In [1]:
import pandas as pd
import inventorize3 as inv

In [2]:
retail_clean = pd.read_csv('retail_clean.csv')
retail_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 779495 entries, 0 to 779494
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      779495 non-null  int64  
 1   StockCode    779495 non-null  object 
 2   Description  779495 non-null  object 
 3   Quantity     779495 non-null  int64  
 4   InvoiceDate  779495 non-null  object 
 5   Price        779495 non-null  float64
 6   Customer ID  779495 non-null  int64  
 7   Country      779495 non-null  object 
 8   Revenue      779495 non-null  float64
dtypes: float64(2), int64(3), object(4)
memory usage: 53.5+ MB


In [3]:
retail_clean['InvoiceDate'] = pd.to_datetime(retail_clean['InvoiceDate'], format = '%d/%m/%Y %H:%M')
retail_clean['date'] = retail_clean['InvoiceDate'].dt.strftime('%Y-%m-%d')
retail_clean['date'] = pd.to_datetime(retail_clean['date'])
retail_clean['year'] = retail_clean['date'].dt.year
years_2 = retail_clean[retail_clean['year'].isin([2010, 2011])]

In [4]:
total = years_2.groupby(by = ['year', 'Description']).agg(total_sales = ('Quantity', 'sum'), average_price = ('Price', 'mean')).reset_index()
total

,year,Description,total_sales,average_price
0,2010,DOORMAT UNION JACK GUNS AND ROSES,167,6.640196
1,2010,3 STRIPEY MICE FELTCRAFT,399,1.941429
2,2010,4 PURPLE FLOCK DINNER CANDLES,200,2.150000
3,2010,ANIMAL STICKERS,169,0.210000
4,2010,BLACK PIRATE TREASURE CHEST,6,1.650000
...,...,...,...,...
8102,2011,ZINC T-LIGHT HOLDER STARS SMALL,4894,0.836975
8103,2011,ZINC TOP 2 DOOR WOODEN SHELF,9,16.950000
8104,2011,ZINC WILLIE WINKIE CANDLE STICK,2432,0.873833
8105,2011,ZINC WIRE KITCHEN ORGANISER,25,7.175000


In [5]:
expected = total.groupby(by = ['Description']).agg(expected_demand = ('total_sales', 'mean'), 
                                                   sd = ('total_sales', 'std'), 
                                                   average_price = ('average_price', 'mean')).reset_index()

def marginal_error(df):
    if pd.isna(df['sd']):
        return df['expected_demand']*0.1
    else:
        return df['sd']

In [6]:
expected['sd1'] = expected.apply(func = marginal_error, axis = 1)
expected

,Description,expected_demand,sd,average_price,sd1
0,DOORMAT UNION JACK GUNS AND ROSES,167.0,NaN,6.640196,16.700000
1,3 STRIPEY MICE FELTCRAFT,399.0,NaN,1.941429,39.900000
2,4 PURPLE FLOCK DINNER CANDLES,162.0,53.740115,2.224286,53.740115
3,50'S CHRISTMAS GIFT BAG LARGE,1885.0,NaN,1.248073,188.500000
4,ANIMAL STICKERS,169.0,NaN,0.210000,16.900000
...,...,...,...,...,...
5146,ZINC T-LIGHT HOLDER STARS SMALL,4894.0,NaN,0.836975,489.400000
5147,ZINC TOP 2 DOOR WOODEN SHELF,69.0,84.852814,14.753571,84.852814
5148,ZINC WILLIE WINKIE CANDLE STICK,2818.0,545.886435,0.860579,545.886435
5149,ZINC WIRE KITCHEN ORGANISER,25.0,NaN,7.175000,2.500000


In [7]:
expected['cost'] = expected['average_price']*0.6

In [8]:
empty_df = pd.DataFrame()

In [9]:
for i in range(len(expected)):
    a = inv.MPN_singleperiod(mean = expected['expected_demand'][i], 
                             standerddeviation = expected['sd1'][i], 
                             p = expected['average_price'][i], 
                             c = expected['cost'][1], g = 0, b =  0)
    b = pd.DataFrame(a, index = [0])
    b['Description'] = expected['Description'][i]
    empty_df = pd.concat([empty_df, b], axis = 0)

C:\Users\thanh\anaconda3\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2241: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
C:\Users\thanh\anaconda3\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2242: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
C:\Users\thanh\anaconda3\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2241: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
C:\Users\thanh\anaconda3\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2242: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
C:\Users\thanh\anaconda3\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2241: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
C:\Users\thanh\anaconda3\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2242: RuntimeWarning: invalid value encountered in mult

In [10]:
empty_df.iloc[1,:]

quantity                          388.891451
demand                                 399.0
sd                                      39.9
unitshort                          21.480197
shortagecost                       41.702268
cost                              453.002984
revenue                               774.63
profit                            279.924748
soldatfullprice                   377.519803
sold_discount                      11.371647
Description         3 STRIPEY MICE FELTCRAFT
Name: 0, dtype: object